In [3]:
# Importing libraries needed for the project
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import linregress
import matplotlib.pyplot as plt
import plotly.express as px
from math import ceil #The math.ceil() method rounds a number UP to the nearest integer, if necessary, and returns the result.
from urllib.request import urlopen
import json
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning

In [4]:
focusareas = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

In [6]:
# Load the Excel file into a DataFrame
df = pd.read_excel("./2minV1-Answers.xlsx")

# Save the DataFrame as a CSV file
df.to_csv("./2minV1-Answers.csv", index=False)

# Display the first few rows (optional, just to check the data)
df.head()


,User Name,Score(%),Attempt Date,User Email,Taken Time,What is your gender?,Male,Female,Other,How old are you?,...,Progesterone,Luteinizing hormone (LH),Estrogen,Follicle-stimulating hormone (FSH),How confident are you in your answers.1,Not at all confident.1,Slightly confident.1,Somewhat confident.1,Quite confident.1,Extremely confident.1
0,Guest,14.285714,11-6-2024 12:43:26 PM,NaN,0:3:27,Incorrect,NaN,Female,NaN,28,...,NaN,Luteinizing hormone (LH),NaN,NaN,Incorrect,NaN,Slightly confident,NaN,NaN,NaN
1,Guest,14.285714,11-6-2024 09:56:16 AM,NaN,0:3:3,Incorrect,NaN,Female,NaN,25,...,NaN,NaN,Estrogen,NaN,Incorrect,NaN,NaN,Somewhat confident,NaN,NaN
2,Guest,14.285714,11-6-2024 09:40:01 AM,NaN,0:4:39,Incorrect,Male,NaN,NaN,26,...,NaN,Luteinizing hormone (LH),NaN,NaN,Incorrect,NaN,NaN,Somewhat confident,NaN,NaN
3,Guest,14.285714,11-6-2024 09:25:58 AM,NaN,0:3:7,Incorrect,Male,NaN,NaN,23,...,NaN,Luteinizing hormone (LH),NaN,NaN,Incorrect,NaN,NaN,NaN,Quite confident,NaN
4,Guest,28.571428,11-6-2024 09:11:37 AM,NaN,0:3:3,Incorrect,Male,NaN,NaN,22,...,NaN,Luteinizing hormone (LH),NaN,NaN,Incorrect,NaN,NaN,NaN,NaN,Extremely confident
